In [ ]:
### Reflecting DB
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func



In [ ]:
# Create the engine to connect to a sqlite file.
# The file will not be created until the class, base, and metadata are executed

engine = create_engine("sqlite:///db/CCES_Ver50.sqlite")

# Create a variable to manage session.queries later on...
session = Session(engine)

In [ ]:
Base = automap_base()

engine = create_engine("sqlite:///db/CCES_Ver51.sqlite")

Base.prepare(engine, reflect=True)
CCES_16 = Base.classes.CCES_16

session = Session(engine)

In [ ]:
# Declare a base using automap_base()
Base = automap_base()
Base

In [ ]:
# Use the base class to reflect the database tables
### Define our Survey Class for the sqlite db table
Base = declarative_base()

class Survey(Base):
    __tablename__ = 'CCES_16'
    index = Column(Integer, primary_key=True)
    V101 = Column(Integer)
    lookupzip = Column(Integer)
    StateName = Column(String(255))
    StateAbb = Column(String(255))
    Latitude = Column(Float)
    Longitude = Column(Float)
    countyfips = Column(Float)
    birthyr = Column(Integer)
    gender = Column(String(255))
    educ = Column(String(255))
    GunBackgroundChecks_16 = Column(String(255))
    ProhibitPublication_16 = Column(String(255))
    BanAssultWeapons_16 = Column(String(255))
    MakeCCPEasier_16 = Column(String(255))
    AlwaysAllowChoice_16 = Column(String(255))
    RapeIncestorHealth_16 = Column(String(255))
    ProhibitMoreThan20Weeks_16 = Column(String(255))
    Employersdeclinebenefits_16 = Column(String(255))
    ProhibitFedFunds_16 = Column(String(255))
    GayMarriage = Column(String(255))

Base.prepare(engine, reflect=True)

In [ ]:
# Print all of the classses mapped to the Base (this provides a list of the table names in the sqlite.db)
Base.classes.keys()

In [ ]:
# Assign the CCES_16 class to a variable called 'CCES'
CCES = Base.classes.CCES_16

In [ ]:
session = Session(engine)

In [ ]:
#############################################
# Dropdown Variables mapped to Sql...
#############################################

# Gun Questions
GunBackgroundChecks_16 = CCES.GunBackgroundChecks_16
ProhibitPublication_16 = CCES.ProhibitPublication_16
BanAssultWeapons_16 = CCES.BanAssultWeapons_16
MakeCCPEasier_16 = CCES.MakeCCPEasier_16

# Abortion Questions
AlwaysAllowChoice_16 = CCES.AlwaysAllowChoice_16
RapeIncestorHealth_16 = CCES.RapeIncestorHealth_16
ProhibitMoreThan20Weeks_16 = CCES.ProhibitMoreThan20Weeks_16
Employersdeclinebenefits_16 = CCES.Employersdeclinebenefits_16
ProhibitFedFunds_16 = CCES.ProhibitFedFunds_16

# Gay Marriage Question
GayMarriage = CCES.GayMarriage
#############################################

# Flask Route - metadata totals testing

In [ ]:
#############################################
# Creating metadata totals function for metadata totals flask route
#############################################

In [ ]:
#############################################
# Function to return Totals - Total Support and Total Oppose for each issue
#############################################
def sample_metadatatotals(sample):
    """Return the totals for a given sample"""
    TotalResults = []
    
    Approve = {}
    Oppose = {}
    
    ApproveTemp = session.query(CCES).filter(sample == 'Support').count()
    Approve['Approve'] = ApproveTemp
    TotalResults.append(Approve)
    
    OpposeTemp = session.query(CCES).filter(sample == 'Oppose').count()
    Oppose['Oppose'] = OpposeTemp
    TotalResults.append(Oppose)

    print(TotalResults)
#     return jsonify(TotalResults)

In [ ]:
#############################################
# Metadatatotals Function Testing 1

sample_metadatatotals(GayMarriage)

In [ ]:
#############################################
# Metadatatotals Function Testing 2

sample_metadatatotals(CCES.GunBackgroundChecks_16)

In [ ]:
#############################################
# Metadatatotals Function - without function - direct query to verify results

ApproveGay = session.query(CCES).filter(CCES.GayMarriage == 'Support').count()
OpposeGay = session.query(CCES).filter(CCES.GayMarriage == 'Oppose').count()
print(f'Sqlite query of votes supporting Gay Marrige: {ApproveGay}')
print(f'Sqlite query of votes opposint Gay Marriage: {OpposeGay}')

# Flask Route - Metadata States Groupby Results

In [ ]:
#############################################
# Metadata States Groupby Function
#############################################

def Metadata_States(sample):
    
    # Query to create two lists of tuples (state, number)
    StatesFor = session.query(CCES.StateName, func.count(sample)).\
                filter(sample == 'Support').\
                group_by(CCES.StateName).all()
    StatesNot = session.query(CCES.StateName, func.count(sample)).\
                filter(sample == 'Oppose').\
                group_by(CCES.StateName).all()

    # List that will hold final dictionaries - to be jsonified
    States_Results = []
    
    for i, j in zip(StatesFor, StatesNot):

        tempfile = {}
        tempfile['State'] = i[0]
        tempfile['Support'] = i[1]
        tempfile['Oppose'] = j[1]
        if i[1] > j[1]:
            tempoverall = 'Support'
        else:
            tempoverall = 'Oppose'
        tempfile['Overall'] = tempoverall

        States_Results.append(tempfile)

    return States_Results
    
            

In [ ]:
#############################################
# Metadata States Function Testing 1

Metadata_States(GunBackgroundChecks_16)

In [ ]:
#############################################
# Filter by state how many respondents were supportive of Gun Background Checks
#############################################

StatesFor = session.query(CCES.StateName, func.count(CCES.GunBackgroundChecks_16)).filter(CCES.GunBackgroundChecks_16 == 'Support').group_by(CCES.StateName).all()
StatesNot = session.query(CCES.StateName, func.count(CCES.GunBackgroundChecks_16)).filter(CCES.GunBackgroundChecks_16 == 'Oppose').group_by(CCES.StateName).all()
print(StatesFor[0:5])
print(StatesNot[0:5])

In [ ]:
#############################################
# Groupby Function testing before building function
#############################################
StatesForAbortion = session.query(CCES.StateName, func.count(CCES.AlwaysAllowChoice_16)).filter(CCES.AlwaysAllowChoice_16 == 'Support').group_by(CCES.StateName).all()
StatesNotAbortion = session.query(CCES.StateName, func.count(CCES.AlwaysAllowChoice_16)).filter(CCES.AlwaysAllowChoice_16 == 'Oppose').group_by(CCES.StateName).all()

ThirdTry = []

for i, j in zip(StatesForAbortion, StatesNotAbortion):

    tempfile = {}
    tempfile['State'] = i[0]
    tempfile['Support'] = i[1]
    tempfile['Oppose'] = j[1]
    if i[1] > j[1]:
        tempoverall = 'Support'
    else:
        tempoverall = 'Oppose'
    tempfile['Overall'] = tempoverall
        
    ThirdTry.append(tempfile)
ThirdTry

In [ ]:
#############################################
# Testing Groupby by specific state
# Filter by state how many respondents were supportive of Gun Background Checks
#############################################
AlabamaTotalExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').count()
AlabamaForExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').filter(CCES.GayMarriage == 'Support').count()
AlabamaAgainstExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').filter(CCES.GayMarriage == 'Oppose').count()
print(AlabamaTotalExample)
print(AlabamaForExample)
print(AlabamaAgainstExample)